In [2]:
%load_ext autoreload
%autoreload 2

# Imports

In [3]:
import os
import pandas as pd

import numpy as np
import jax
jax.config.update('jax_platform_name', 'gpu')



In [4]:
fn_EcoCyc = '../data/sRNA/EcoCyc/EcoCyc_regulatory_RNAs.csv'
ecocyc = pd.read_csv(fn_EcoCyc)
fn_RNAInter = os.path.join('..', 'data', 'sRNA', 'RNAInter', 'Download_data_RR.csv')
rnainter = pd.read_csv(fn_RNAInter)
# data = pd.concat([pd.read_csv(fn_RNAInter), pd.read_csv(fn_EcoCyc)])
try:
    rnainter = rnainter.drop(columns=['Unnamed: 0'])
except:
    pass


### Start with RNAInter

In [5]:
data = pd.DataFrame()
rnainter['Database'] = 'RNAInter'
for n, o in zip(['Name', 'ID', 'Category', 'Symbol', 'Sequence', 'Database'], ['Interactor1.Symbol', 'Raw_ID1', 'Category1', 'Interactor1.Symbol', 'Sequence1', 'Database']):
    data[n] = rnainter.drop_duplicates(subset=['Interactor1.Symbol'])[o]

data = pd.concat([data, rnainter.drop_duplicates(subset=['Interactor2.Symbol']).reset_index()[['Interactor2.Symbol', 'Raw_ID2', 'Category2', 'Sequence2']].rename(
    columns={j: i for i, j in zip(['Name', 'ID', 'Category', 'Symbol', 'Sequence'], ['Interactor2.Symbol', 'Raw_ID2', 'Category2', 'Interactor2.Symbol', 'Sequence2'])})], axis=0)

data = data[~data['Name'].isna()]
data = data.drop_duplicates(subset=['Name'])

In [6]:
data['Symbol'].value_counts()


Symbol
acnA    1
metH    1
malK    1
manX    1
manY    1
       ..
fumC    1
fur     1
gadB    1
gadX    1
symE    1
Name: count, Length: 154, dtype: int64

### Now EcoCyc

In [7]:
ecocyc.head()

,Ontology - all instances of class from My Favorites,Matches,Common-Name,GO-Terms,Names,Object ID,Gene,Sequence - DNA sequence,Copy-Number,Common-Name.1
0,6S-RNA,Regulatory-RNAs,6S RNA,NaN,6S RNA // SsrS // Ssr,6S-RNA,EG30099,ATTTCTCTGAGATGTTCGCAAGCGGGCCAGTCCCCTGAGCCGATAT...,NaN,ssrS
1,RDLD-RNA,Regulatory-RNAs,antisense regulatory RNA RdlD,GO:2000766 // GO:0030371,antisense regulatory RNA RdlD // RdlD // RdlD ...,RDLD-RNA,G0-9042,GTCTAGAGTCAAGATTAGCCCCCGTGGTGTTGTCAGGTGCATACCT...,NaN,rdlD
2,RNA0-441,Regulatory-RNAs,<i>arfA</i>-derived small regulatory RNA,GO:0006970 // GO:0003729,<i>arfA</i>-derived small regulatory RNA,RNA0-441,G0-17130,AAGAAAGTAAATCACTTTTTTACCACTGGTCTTCTGCTTTCAGGCG...,NaN,yrdG
3,RNA0-409,Regulatory-RNAs,e14 prophage; small regulatory RNA C0293,GO:0035194 // GO:0040033 // GO:0003729,e14 prophage; small regulatory RNA C0293 // C0293,RNA0-409,G0-8897,TACTCACCCGGGACTCGCCAGGGGACAGCCAACAGGCATTGGGTGC...,NaN,C0293
4,RNA0-162,Regulatory-RNAs,putative antisense regulatory RNA RdlB,GO:1902201,putative antisense regulatory RNA RdlB // RdlB,RNA0-162,G0-9606,GTCTGGTTTCAAGATTAGCCCCCGTTCTGTTGTCAGGTTTTACCTC...,NaN,rdlB


In [8]:
ecocyc['Category'] = 'sRNA'
ecocyc['Database'] = 'EcoCyc'
ecocyc = ecocyc.drop_duplicates(subset=['Common-Name'])

data = pd.concat([data, ecocyc.rename(columns={j: i for i, j in zip(['Name', 'ID', 'Symbol', 'Sequence'], ['Common-Name', 'Object ID', 'Common-Name.1', 'Sequence - DNA sequence'])})[
    ['Name', 'ID', 'Category', 'Symbol', 'Sequence', 'Database']]], axis=0)

# Delete duplicates responsibly

In [9]:
doubles = data['Symbol'].value_counts()[data['Symbol'].value_counts() > 1].index.to_list()

for d in doubles:
    
    if len(data[data['Symbol'] == d]['Sequence'].unique()) > 1:
        print('Not removing duplicate of ', d, data[data['Symbol'] == d]['Name'].unique())
        
    else:
        print(d)
        print(data[data['Symbol'] == d]['Sequence'].values)
    
data[data['Symbol'] == d].head()
data.drop(index= data[data['Symbol'] == d].index[0])
# data[data['Symbol'] == d].index[0]

Not removing duplicate of  micA ['micA' 'small regulatory RNA MicA']
Not removing duplicate of  cyaR ['cyaR' 'small regulatory RNA CyaR']
Not removing duplicate of  gadY ['gadY' 'small regulatory RNA GadY']
Not removing duplicate of  micC ['micC' 'small regulatory RNA MicC']
Not removing duplicate of  sgrS ['sgrS' 'small regulatory RNA SgrS']
Not removing duplicate of  sdsR ['sdsR' 'small regulatory RNA SdsR']
Not removing duplicate of  ryhB ['ryhB' 'small regulatory RNA RyhB']
Not removing duplicate of  rybB ['rybB' 'small regulatory RNA RybB']
Not removing duplicate of  rprA ['rprA' 'small regulatory RNA RprA']
Not removing duplicate of  fnrS ['fnrS' 'small regulatory RNA FnrS']
Not removing duplicate of  oxyS ['oxyS' 'small regulatory RNA OxyS']
Not removing duplicate of  omrB ['omrB' 'small regulatory RNA OmrB']
Not removing duplicate of  omrA ['omrA' 'small regulatory RNA OmrA']
Not removing duplicate of  istR ['istR' 'small regulatory RNA IstR-1']
Not removing duplicate of  ohsC 

,Name,ID,Category,Symbol,Sequence,Database
0,acnA,NCBI:946724,mRNA,acnA,ATTCGGAACGAGGCCTGAAGCAGTGTCGCCGTCCCTCTGCCTTGCA...,RNAInter
1,acrZ,NCBI:945365,mRNA,acrZ,ACTTACTACTGTCTTCGGGGGGTCCGAGGTTTCTGGGGGGTCGTAC...,RNAInter
4,argR,NCBI:947861,mRNA,argR,GACAATGGCGATAGTATGGCGGTTGTTTCTTTCCCATCTCTACTCA...,RNAInter
5,arrS,NCBI:11115378,sRNA,arrS,CTCTCCCTCTCTTTCTCTCTTCTCCGCGGCGATACGATCCGAGATG...,RNAInter
6,asr,NCBI:945103,mRNA,asr,TGCTCCTTCTGCTGATGCCCCCATGTTTGTGATGGGCGTGAACCAT...,RNAInter
...,...,...,...,...,...,...
70,small regulatory RNA SroC,RNA0-124,sRNA,sroC,ACTAATTACAAGAACCAGGGGCGGAAATTCCAGCCCTCTCGATTGT...,EcoCyc
71,small regulatory RNA ZbiJ,RNA0-417,sRNA,zbiJ,TTTCCCTGTCTGTTTGCCGACAGACGCATATGCTCTAACCCTCATT...,EcoCyc
72,small RNA SibB,RYED-RNA,sRNA,sibB,GAGGGTAGAGCGGGGTTTCCCCCGCCCTGGTAGTCTTAGTAAGCGG...,EcoCyc
73,small RNA SibD,C0730-RNA,sRNA,sibD,ACAAGGGTGAGGGAGGATTTCTCCCCCCTCTGATTGGCTGTTAATA...,EcoCyc


In [11]:
data.drop_duplicates(subset=['Symbol'])

,Name,ID,Category,Symbol,Sequence,Database
0,acnA,NCBI:946724,mRNA,acnA,ATTCGGAACGAGGCCTGAAGCAGTGTCGCCGTCCCTCTGCCTTGCA...,RNAInter
1,acrZ,NCBI:945365,mRNA,acrZ,ACTTACTACTGTCTTCGGGGGGTCCGAGGTTTCTGGGGGGTCGTAC...,RNAInter
2,arcZ,NCBI:2847690,sRNA,arcZ,CTCATGTTGACCGCTTGTTTAGCAGCTTCAAGGAAGCTGAAGGGCA...,RNAInter
4,argR,NCBI:947861,mRNA,argR,GACAATGGCGATAGTATGGCGGTTGTTTCTTTCCCATCTCTACTCA...,RNAInter
5,arrS,NCBI:11115378,sRNA,arrS,CTCTCCCTCTCTTTCTCTCTTCTCCGCGGCGATACGATCCGAGATG...,RNAInter
...,...,...,...,...,...,...
70,small regulatory RNA SroC,RNA0-124,sRNA,sroC,ACTAATTACAAGAACCAGGGGCGGAAATTCCAGCCCTCTCGATTGT...,EcoCyc
71,small regulatory RNA ZbiJ,RNA0-417,sRNA,zbiJ,TTTCCCTGTCTGTTTGCCGACAGACGCATATGCTCTAACCCTCATT...,EcoCyc
72,small RNA SibB,RYED-RNA,sRNA,sibB,GAGGGTAGAGCGGGGTTTCCCCCGCCCTGGTAGTCTTAGTAAGCGG...,EcoCyc
73,small RNA SibD,C0730-RNA,sRNA,sibD,ACAAGGGTGAGGGAGGATTTCTCCCCCCTCTGATTGGCTGTTAATA...,EcoCyc
